# Day 7
You land at the regional airport in time for your next flight. In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to issues in luggage processing.

Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!

For example, consider the following rules:
```
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
```
These rules specify the required contents for 9 bag types. In this example, every faded blue bag is empty, every vibrant plum bag contains 11 bags (5 faded blue and 6 dotted black), and so on.

You have a shiny gold bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one shiny gold bag?)

In the above rules, the following options would be available to you:

- A bright white bag, which can hold your shiny gold bag directly.
- A muted yellow bag, which can hold your shiny gold bag directly, plus some other bags.
- A dark orange bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny - gold bag.
- A light red bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.

So, in this example, the number of bag colors that can eventually contain at least one shiny gold bag is 4.

How many bag colors can eventually contain at least one shiny gold bag? (The list of rules is quite long; make sure you get all of it.)

In [1]:
import re

**Example Case**

In [2]:
example_rules = """light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags."""

example_rules = example_rules.split('\n')

**Read in Input**

In [3]:
with open('input.txt', 'r') as f:
    rules = f.read().split('\n')

## Part 1

In [4]:
def look_for_bag(current_bag, looking, bag_rules):
    """ look for a specific bag. return 1 if found and 0 if not
    current_bag: str
    looking: str
    bag_rules: dict
    """
    if current_bag is None:
        return 0
    elif current_bag == looking:
        return 1
    else:
        for child in bag_rules[current_bag]:
            if look_for_bag(child, looking, bag_rules) == 1:
                return 1 # if found
        return 0 # if not found
    
def build_rules(rules):
    """ build dictionary to represent rules
    rules: list
    """
    bag_rules = dict()
    for rule in rules:
        bag, rule = rule.split(' contain ')

        # process rule

        rule = rule.replace('.', '').split(', ')
        inside_rules = dict()
        for r in rule:
            if r == 'no other bags':
                number = 0
                inside_bag = None
            else:
                number = int(re.findall(r'\d+', r)[0])
                inside_bag = ' '.join((re.findall(r'\w+', r)[1:]))
                if number == 1:
                    inside_bag += 's'

            inside_rules[inside_bag] = number

        # save rule
        bag_rules[bag] = inside_rules
    return bag_rules
    
def find_bag_with_rules(rules, looking='shiny gold bags'):
    """ find how many combinations exist to have the looking bag inside at least one bag
    rules: list
    looking: str
    """
    bag_rules = build_rules(rules)
    found = -1 # will find it in the actual bag
    for start in bag_rules:
        found += look_for_bag(start, looking, bag_rules)
    return found

assert find_bag_with_rules(example_rules, 'shiny gold bags') == 4

In [5]:
find_bag_with_rules(rules, 'shiny gold bags')

229

## Part 2

It's getting pretty expensive to fly these days - not because of ticket prices, but because of the ridiculous number of bags you need to buy!

Consider again your shiny gold bag and the rules from the above example:

- faded blue bags contain 0 other bags.
- dotted black bags contain 0 other bags.
- vibrant plum bags contain 11 other bags: 5 faded blue bags and 6 dotted black bags.
- dark olive bags contain 7 other bags: 3 faded blue bags and 4 dotted black bags.

So, a single shiny gold bag must contain 1 dark olive bag (and the 7 bags within it) plus 2 vibrant plum bags (and the 11 bags within each of those): 1 + 1 X 7 + 2 + 2 X 11 = 32 bags!

Of course, the actual rules have a small chance of going several levels deeper than this example; be sure to count all of the bags, even if the nesting becomes topologically impractical!

Here's another example:
```
shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.
```
In this example, a single shiny gold bag must contain 126 other bags.

How many individual bags are required inside your single shiny gold bag?

In [6]:
def count_all_bags(current_bag, bag_rules):
    """ count all bags that you'd have to take if you wanted to take your current bags
    current_bag: str
    bag_rules: dict
    """
    if current_bag is None:
        return 0
    else:
        x = 0
        for child, val in bag_rules[current_bag].items():
            total = 0
            total += count_all_bags(child, bag_rules)
            x += val + val * total
        return x

assert count_all_bags('shiny gold bags', build_rules(example_rules)) == 32

In [7]:
count_all_bags('shiny gold bags', build_rules(rules))

6683